<a href="https://colab.research.google.com/github/PRichardRamos/Assistente_TCC_Alura/blob/main/Assistente_de_TCC_Projeto_Alura.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Baixando tudo que será necessário
!pip install -q -U google.generativeai
!pip install -q -U PyPDF2
!pip install -q -U gdown

In [ ]:
# Importanto as bibliotecas
import google.generativeai as genai
from google.colab import userdata
from gdown import download
import PyPDF2
import re

# Chave de API na aba Secrets do Colab
api_code_key = userdata.get('api_key')
genai.configure(api_key=api_code_key)

In [ ]:
# Verificando os modelos disponíveis
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

In [ ]:
# Criando e configurando o modelo de IA
generation_config = {
  "temperature": 1,
  "top_p": 0.95,
  "top_k": 64,
  "max_output_tokens": 8192,
  "response_mime_type": "text/plain",
}
safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE",
  },
]

model = genai.GenerativeModel(
  model_name="gemini-1.5-pro-latest",
  safety_settings=safety_settings,
  generation_config=generation_config,
  system_instruction="Aja como um professor da graduação orientando o aluno em seu TCC. Você lerá um texto acadêmico.\nCaso o tema e a área de atuação do TCC não fique claro, pergunte ao usuário do que se trata.\n\nPrimeiro leia o documento e analise a coesão do texto. Faça sugestões de correção nas partes necessárias, incluindo questões como pontuação e acentuação de palavras.\nComente sobre o tema do TCC e aborde possíveis caminhos para o que falta para ser concluído. Em seguida, escreva um esqueleto para as partes que faltam, a fim de que o aluno apenas preencha as lacunas com as informações necessárias do seu tema.\nCrie uma lista de etapas para a conclusão, tabelando o que falta e mostrando o que pode ser feito para concluir.",
)

In [ ]:
# O usuário insere o link para o PDF no Google Drive
link = input("Me envie seu TCC\n \nInsira aqui o link de compartilhamento seu TCC em pdf\n \nÉ importante que seja um link do Google Drive \n \n")

# Utilize a expressão regular para encontrar o ID do arquivo
id_arquivo = re.search(r'/d/(.+?)/', link).group(1)

# Baixando o TCC para ser analizado
id = id_arquivo
download(id=id, output='tcc.pdf')

# Abrir o arquivo PDF e extrair o texto
with open('/content/tcc.pdf', 'rb') as f:
  pdf_reader = PyPDF2.PdfReader(f)
  text = ''
  for page in pdf_reader.pages:
    text += page.extract_text()


In [ ]:
# Interagir com o modelo de linguagem
chat_session = model.start_chat(
  history=[
    # Você pode adicionar mensagens anteriores aqui
  ]
)
response = chat_session.send_message(text)
print(response.text)